In [5]:
import pandas as pd
import numpy as np

import lightgbm as lgb
import catboost as cat

from sklearn.metrics import *
from sklearn.model_selection import *

from sklearn import preprocessing
le = preprocessing.LabelEncoder()


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [27]:
data=pd.concat([train,test])
data.drop(['id'],axis=1,inplace=True)

cat_col=['Gender', 'Driving_License', 'Region_Code','Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage','Policy_Sales_Channel']
for col in cat_col:
    data[col]=le.fit_transform(data[col])
    
train2=data[~data['Response'].isna()]
test2=data[data['Response'].isna()]

X_train=train2.drop(['Response'],axis=1)
y=train2['Response']
X_test=test2[X_train.columns]
cat_cols=[X_train.columns.get_loc(c) for c in cat_col]

In [69]:
clf_lgb = lgb.LGBMClassifier(boosting_type='gbdt', 
                      objective='binary',
                      n_estimators=500, 
                      learning_rate=0.1,
                      categorical_feature = cat_cols,
                      scale_pos_weight=7,
                      random_state=101
                     )

In [70]:
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=101)
cv_score = []
pred_test_lgb =np.zeros((len(X_test),1))
feat_imp=np.zeros((len(X_train.columns),1))

for train_index,test_index in skf.split(X_train,y):
    x_train,x_val = X_train.iloc[train_index],X_train.iloc[test_index]
    y_train,y_val = y.iloc[train_index],y.iloc[test_index]
    clf = clf_lgb
    clf.fit(x_train,y_train)
    score = round(roc_auc_score(y_val,clf.predict(x_val)),2)
    cv_score.append(score)
    print(score,end=",")
    
    #predictions
    pred_test_lgb += clf.predict_proba(X_test)[:,1].reshape(-1,1)
    feat_imp += clf.feature_importances_.reshape(-1,1)
    
pred_test_lgb = pred_test_lgb/5
feat_imp = feat_imp/5

feature_imp = pd.DataFrame(sorted(zip(X_train.columns,feat_imp)), columns=['Feature','Value'])
feature_imp=feature_imp.sort_values(by='Value',ascending=False)

print(np.mean(cv_score))

0.8,0.79,0.79,0.79,0.79,0.792


In [72]:
sub=pd.DataFrame()
sub['Response']=pred_test_lgb.ravel()
sub.index=test.id
sub=sub[['Response']]
sub['Response']=np.where(sub['Response']>=0.5,1,0)
sub.to_csv('try1.csv')
sub.head()

,Response
id,
381110,0
381111,1
381112,1
381113,0
381114,0
